# Fn Guide 데이터 사용을 넣어주는 이유
- 위에서 한 Marcap을 이용해서 가져온 데이터는 수정주가가 아니고 상장 주식수를 이용해서 한번 전처리를 했어도 회사 병합, 배당 등등 의 사항들이 수정 주가에 들어가 있지 않는다.
- Fn Guide에서는 현금 배당을 포함한 수정주가 데이터를 가져올 수 있고 상장폐지 데이터도 가져올 수 있어서 Fn Guide 구독이 되어 있으면 Fn Guide에서 데이터를 가져오는 것을 추천한다.

In [1]:
import pandas as pd 
from pathlib import Path

In [2]:
idx = pd.IndexSlice

## Fn Guide 엑셀 데이터 형식 멀티 인덱스로 변환 함수 정의

In [3]:
def cleaning_fn_data(data: pd.DataFrame, column_name) -> pd.DataFrame:
    '''
    Transform FN Guide Excel Data to Multi Index DataFrame
    input
    data pd.DataFrame:
        FN Guide Excel Data Format
    
    column_name str:
        Set DataFrame column name
    
    return -> pd.DataFrame (eg. SamSung Equity)
    
                         column_name
    date        ticker 
    2017-01-01  005930       19950
    2017-01-02  005930       19350
    2017-01-04  005930       20050
    2017-01-05  005930       21050
    '''
    tickers_list = data.iloc[7].to_list()
    tmp = data.iloc[13:]
    tickers_index = pd.Index(data=tickers_list, name='ticker')
    tmp.columns = tickers_index
    tmp.set_index('Symbol', inplace=True)
    tmp.index.name = 'date'
    data =  tmp.rename(columns=lambda x: x.replace('A', '')).stack('ticker').to_frame(column_name) 
    return data.astype({column_name: 'int64'})

## 수정시가 가져오기

In [4]:
adj_open = pd.read_excel('FNGuide/수정시가_현금배당.xlsx')

In [5]:
adj_open_cleaning = cleaning_fn_data(adj_open, 'adj_open')

In [6]:
adj_open_cleaning

adj_open
date       ticker          
1990-01-01 005930       280
           006400     14667
           005380      9713
           000270    148637
           005490     13435
...                     ...
2021-07-26 012600      2490
           001770     26350
           002420      6160
           008500     21050
           010580       306

[5836780 rows x 1 columns]

## 수정종가 가져오기

In [7]:
adj_close = pd.read_excel('FNGuide/수정종가_현금배당.xlsx')

In [8]:
adj_close_cleaning = cleaning_fn_data(adj_close, 'adj_close')

## 수정고가 가져오기

In [9]:
adj_high = pd.read_excel('FNGuide/수정고가_현금배당.xlsx')

In [10]:
adj_high_cleaning = cleaning_fn_data(adj_high, 'adj_high')

## 수정저가 가져오기

In [11]:
adj_low = pd.read_excel('FNGuide/수정저가_현금배당.xlsx')

In [12]:
adj_low_cleaning = cleaning_fn_data(adj_low, 'adj_low')

## 거래량 가져오기

In [13]:
volume = pd.read_excel('FNGuide/거래량.xlsx')

In [14]:
volume_cleaning = cleaning_fn_data(volume, 'volume')

## 상장주식수 가져오기

In [15]:
list_shares = pd.read_excel('FNGuide/상장주식수.xlsx')

In [16]:
list_shares_cleaning = cleaning_fn_data(list_shares, 'stocks')

## 시가총액 가져오기

In [17]:
marcap = pd.read_excel('FNGuide/시가총액.xlsx')

In [18]:
marcap_cleaning = cleaning_fn_data(marcap, 'marcap')

## 데이터 확인

In [130]:
returns = adj_close_cleaning.unstack('ticker').pct_change(240).stack('ticker')

In [131]:
buffer = 0.00001

In [132]:
lower_return = returns['adj_close'].quantile(buffer)
upper_return = returns['adj_close'].quantile(1-buffer)

In [133]:
upper_return

4499.0

In [134]:
# lower_tickers = returns[returns['adj_close'] < lower_return].index.get_level_values('ticker').unique()
upper_tickers = returns[returns['adj_close'] > upper_return].index.get_level_values('ticker').unique()

In [135]:
# intersact_tickers = lower_tickers.union(upper_tickers)

In [136]:
upper_tickers

Index(['008080'], dtype='object', name='ticker')

In [142]:
upper_tickers.intersection(exclude_ticker)

Index(['008080'], dtype='object')

### 상하한가 제한이 있는데 이상해 보일 수 있지만 상폐전에 정리매매에서는 상하한가 제한이 없기 때문에 제대로 된 데이터 인것을 알 수 있다.

In [144]:
datas = pd.concat([adj_high_cleaning, adj_low_cleaning, adj_open_cleaning, adj_close_cleaning, volume_cleaning], axis=1)

In [145]:
datas.isna().sum()

adj_high       0
adj_low        0
adj_open       0
adj_close      0
volume       789
dtype: int64

## 거래량이 결측치가 많은것을 알 수 있고 거래량의 평균값으로 결측치를 대체한다.

In [146]:
volume_data = datas[['volume']]

In [147]:
tickers = volume_data.index.get_level_values('ticker').unique()

In [148]:
volume_data_list = []
for i, ticker in enumerate(tickers):
    if i % 100 == 0:
        print(i)
    data = volume_data.loc[idx[:, ticker], :]
    data = data.fillna(data.mean())
    volume_data_list.append(data)

0
100
200
300
400
500
600
700
800
900
1000
1100


In [149]:
volume_fillna_data = pd.concat(volume_data_list).sort_index()

In [150]:
datas['volume'] = volume_fillna_data['volume']

## Ticker들과 자산이름 데이터 생성

In [176]:
tickers = adj_close.iloc[7].apply(lambda x: x.replace('A', '')).to_list()

In [177]:
tickers

['Symbol',
 '005930',
 '000660',
 '035420',
 '035720',
 '207940',
 '051910',
 '006400',
 '005380',
 '068270',
 '000270',
 '005490',
 '066570',
 '012330',
 '028260',
 '096770',
 '051900',
 '017670',
 '105560',
 '055550',
 '034730',
 '036570',
 '011200',
 '361610',
 '015760',
 '003550',
 '032830',
 '018260',
 '090430',
 '009150',
 '086790',
 '251270',
 '003670',
 '302440',
 '034020',
 '352820',
 '033780',
 '010950',
 '003490',
 '000810',
 '010130',
 '326030',
 '009540',
 '011170',
 '030200',
 '018880',
 '009830',
 '034220',
 '316140',
 '024110',
 '086280',
 '097950',
 '004020',
 '011780',
 '032640',
 '021240',
 '071050',
 '161390',
 '000720',
 '011790',
 '137310',
 '006800',
 '035250',
 '011070',
 '267250',
 '008930',
 '002790',
 '241560',
 '383220',
 '271560',
 '028050',
 '180640',
 '139480',
 '000100',
 '000120',
 '010140',
 '078930',
 '029780',
 '028670',
 '003410',
 '004990',
 '128940',
 '016360',
 '005830',
 '020150',
 '006360',
 '298020',
 '112610',
 '007070',
 '006280',
 '008770',

In [178]:
names = adj_close.iloc[8].to_list()

In [184]:
company_data = pd.DataFrame({tickers[0]:tickers[1:], names[0]:names[1:]})

In [185]:
company_data.columns = ['ticker', 'name']

In [186]:
company_data

,ticker,name
0,005930,삼성전자
1,000660,SK하이닉스
2,035420,NAVER
3,035720,카카오
4,207940,삼성바이오로직스
...,...,...
1172,900030,연합과기
1173,900050,중국원양자원
1174,950010,평산차업 KDR
1175,950070,중국고섬


## 이상한 종목들 제거

### 백테스트 경험을 토대로 제외해야되는 종목코드 추가

In [159]:
exclude_tickers = pd.Index(['025620', 
                           '001260',
                           '015540',
                           '003570',
                           '008080',
                           '003060',
                           '017170',
                           '002540',
                           '008400',
                           '009310',
                          ]).unique()

In [158]:
tickers = datas.index.get_level_values('ticker').unique()

In [160]:
cleaning_tickers = tickers.difference(exclude_tickers)

In [161]:
cleaning_tickers

Index(['000010', '000020', '000030', '000040', '000050', '000060', '000070',
       '000080', '000090', '000100',
       ...
       '380440', '383220', '383800', '900030', '900050', '900140', '950010',
       '950070', '950100', '950210'],
      dtype='object', length=1161)

In [163]:
# 시간 오래 걸립니다.
datas = datas.loc[idx[:, cleaning_tickers], :]

In [167]:
company_data

,ticker,name
0,005930,삼성전자
1,000660,SK하이닉스
2,035420,NAVER
3,035720,카카오
4,207940,삼성바이오로직스
...,...,...
1172,900030,연합과기
1173,900050,중국원양자원
1174,950010,평산차업 KDR
1175,950070,중국고섬


In [187]:
company_data.set_index('ticker', inplace=True)

In [188]:
company_data = company_data.reindex(index=cleaning_tickers).reset_index()

In [193]:
company_data.columns = ['ticker', 'name']

## Data 저장

In [24]:
DATA_STORE = Path('fnguide_data.h5')

In [197]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('fnguide/prices', datas)
    store.put('fnguide/stocks', company_data)
    store.put('fnguide/marcap', marcap_cleaning)
    store.put('fnguide/share', list_shares_cleaning)

In [26]:
with pd.HDFStore(DATA_STORE) as store:
    print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: fnguide_data.h5
/fnguide/marcap            frame        (shape->[5836780,1])
/fnguide/prices            frame        (shape->[5768127,5])
/fnguide/share             frame        (shape->[5832145,1])
/fnguide/stocks            frame        (shape->[1,2])      


In [27]:
list_shares_cleaning

stocks
date       ticker          
1990-01-01 005930  33200000
           006400   5400000
           005380  34881628
           000270  30000000
           005490  91789134
...                     ...
2021-07-23 012600  13589892
           001770   1214878
           002420   4075920
           008500   1200000
           010580  64581636

[5832145 rows x 1 columns]